## Coding Interview Coach  
Example of creating an assistant for coding interviews with the new OpenAI Assistants API.

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

### Creating the Assistant

In [3]:
load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPEN_AI_API_KEY")
)

instructions = """You are an experienced coding interview coach that will help a junior software
developer for an interview. Your main tasks will be:
- Give problems on most common topics in coding interviews and return some examples.
- When asked to evaluate a solution (in any programming language) for a problem, evaluate the 
  correctness of the solution running the code over a sample of inputs."""

# Creating the agent
assistant = client.beta.assistants.create(
    name="Personal Coding Interview Coach",
    instructions=instructions,
    model="gpt-3.5-turbo",
    tools=[{"type": "code_interpreter"}]
)


### Creating a thread

In [4]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_JBmhDZmLqjVOZf09h9YbwpN1', created_at=1700410886, metadata={}, object='thread')


### Adding a message to the thread

In [6]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Give me an easy problem with arrays"
)
print(message.json())

{"id": "msg_m79VrxSgSiSNcFJJXD3Ort8Q", "assistant_id": null, "content": [{"text": {"annotations": [], "value": "Give me an easy problem with arrays"}, "type": "text"}], "created_at": 1700412050, "file_ids": [], "metadata": {}, "object": "thread.message", "role": "user", "run_id": null, "thread_id": "thread_JBmhDZmLqjVOZf09h9YbwpN1"}


### Run the assistant

In [7]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Ryan Gosling. The user has a premium account."
)
print(run)

Run(id='run_A9EMAXzrgUgSSJkyuVKaF59w', assistant_id='asst_YVgo81fA4z8uZSCeKGhpABtz', cancelled_at=None, completed_at=None, created_at=1700412670, expires_at=1700413270, failed_at=None, file_ids=[], instructions='Please address the user as Ryan Gosling. The user has a premium account.', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_JBmhDZmLqjVOZf09h9YbwpN1', tools=[ToolAssistantToolsCode(type='code_interpreter')])


### Check Run status

In [8]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(run)

Run(id='run_A9EMAXzrgUgSSJkyuVKaF59w', assistant_id='asst_YVgo81fA4z8uZSCeKGhpABtz', cancelled_at=None, completed_at=1700412674, created_at=1700412670, expires_at=None, failed_at=None, file_ids=[], instructions='Please address the user as Ryan Gosling. The user has a premium account.', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=1700412670, status='completed', thread_id='thread_JBmhDZmLqjVOZf09h9YbwpN1', tools=[ToolAssistantToolsCode(type='code_interpreter')])


### Display the Assistant's response

In [9]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

# Printing from oldest messages to the newest ones
for message in reversed(messages.data):
    print(f"{message.role}: {message.content[0].text.value}")

user: Give me an easy problem with arrays
user: Give me an easy problem with arrays
assistant: Sure, how about this:

Problem: Find the maximum element in an array.

Write a function `find_max(arr)` that takes in an array `arr` as input and returns the maximum element in the array.

For example, given the input array `[3, 9, 2, 5, 1]`, the function should return `9` since `9` is the largest element in the array.

To solve this problem, you can initialize a variable `max_num` to the first element of the array. Then, iterate through the remaining elements of the array and update `max_num` if you find a larger number.

Please go ahead and write the code to solve this problem.


### Solving the problem

In [10]:
answer = """Here is my answer in python code is it right?:
```
def find_max(arr):
    if len(arr) == 0:
        return 
    maximum = arr[0]
    for num in arr:
        if num > maximum:
            maximum = num
    return maximum
```
"""
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=answer
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please evaluate this code, test it with corner cases as well."
)

In [14]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

# Printing the last response from the assistant
print(f"{messages.data[0].role}: {messages.data[0].content[0].text.value}")

assistant: Your code passed all the test cases, including corner cases. Well done! The function correctly finds the maximum element in the array.

Here are the outputs for the test cases:
- Test Case 1: `9`
- Test Case 2: `None`
- Test Case 3: `-1`
- Test Case 4: `8`
- Test Case 5: `7`

Is there anything else I can help you with?
